In [1]:
import numpy as np
import pylab as pl
import cosmocnc as cnc
import time
from astropy.io import fits

2025-04-13 16:18:08.124688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744553888.138034 4112732 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744553888.141670 4112732 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


defaulting to  /rds-d4/user/iz221/hpc-work/cosmopower/
defaulting to  /home/anicola_hpc/projects/ACT_clustering/code/cosmocnc/cosmocnc/../


In [2]:
zc_edges = np.array([0.2, 0.4, 0.55, 0.7, 0.85])

In [10]:
number_counts = cnc.cluster_number_counts()

number_counts.cnc_params["survey_sr"] = "/home/anicola_hpc/projects/ACT_clustering/scripts/survey_sr_act_DES.py"
number_counts.cnc_params["survey_cat"] = None

number_counts.cnc_params["load_catalogue"] = False

number_counts.cnc_params["cluster_catalogue"] = "act"
number_counts.cnc_params["observables"] =  [["q_act"]]
number_counts.cnc_params["obs_select"] = "q_act"

number_counts.cnc_params["likelihood_type"] = "unbinned"
number_counts.cnc_params["data_lik_from_abundance"] = True
number_counts.cnc_params["compute_abundance_matrix"] = True

number_counts.cnc_params["obs_select_min"] = 5.
number_counts.cnc_params["obs_select_max"] = 200.
number_counts.cnc_params["M_min"] = 5e13
number_counts.cnc_params["M_max"] = 1e16
number_counts.cnc_params["z_min"] = 0.2
number_counts.cnc_params["z_max"] = 0.85

number_counts.cnc_params["n_points"] = 1024*16
number_counts.cnc_params["n_z"] = 100

number_counts.cnc_params["scalrel_type_deriv"] = "numerical"
number_counts.cnc_params["abundance_integral_type"] = "fft"
number_counts.cnc_params["apply_obs_cutoff"] = False

number_counts.cnc_params["cosmology_tool"] = "classy_sz"
number_counts.cnc_params["hmf_calc"] = "classy_sz"
number_counts.cnc_params["class_sz_hmf"] = "T08M200c"
number_counts.cnc_params["mass_definition"] = "200c"
number_counts.cnc_params["hmf_type_deriv"] = "numerical"
# number_counts.cnc_params["interp_tinker"] = "linear"


number_counts.cnc_params["number_cores_hmf"] = 1
number_counts.cnc_params["number_cores_abundance"] = 1#8
number_counts.cnc_params["number_cores_data"] = 1# 8
number_counts.cnc_params["parallelise_type"] = "redshift"

number_counts.cnc_params["cosmo_amplitude_parameter"] = "sigma_8"
number_counts.cnc_params["Hubble_parameter"] = "H0"
number_counts.cnc_params["cosmo_param_density"] = "physical"
number_counts.cnc_params["class_sz_cosmo_model"] = "mnu"
number_counts.cnc_params["cosmo_model"] = number_counts.cnc_params["class_sz_cosmo_model"]

#    "class_sz_cosmo_model": "lcdm", # lcdm, mnu, neff, wcdm, ede

number_counts.cnc_params["cosmocnc_verbose"] = "minimal"

number_counts.cnc_params["likelihood_type"] = "binned"
number_counts.cnc_params["obs_select_min"] = 5.000

number_counts.scal_rel_params["q_cutoff"] = 0.

number_counts.cnc_params["bins_edges_obs_select"] = np.exp(np.linspace(np.log(number_counts.cnc_params["obs_select_min"]), \
                                                                    np.log(number_counts.cnc_params["obs_select_max"]),7))

number_counts.scal_rel_params.update({
'A0' : np.log10(2.2e-05),
'B0': 0.08,
'SZmPivot' : 4.25e14, #in Msun
'bias_sz' : 1.,
'sigma_lnq_act' : 0.2,
'C0' : 2.,
"dof":0.,
}
)

number_counts.cosmo_params.update({

"Ob0h2":0.049*0.68**2,
"Oc0h2":(0.261)*0.68**2,
'H0': 68.,
'h': 0.68,
'sigma8': 0.81,
'tau_reio':  0.0561,
'n_s': 0.965,
"m_nu":0.000006,
"A_s":2.080447023199556e-09,
}
)

In [11]:
number_counts.initialise()

number_counts.get_number_counts() #Evaluate the binned likelihood

n_tot = number_counts.n_tot

print(n_tot)

INFO - Cosmology params: {'Om0': 0.31, 'Ob0': 0.04900000000000001, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.080447023199556e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.0013931690968152894, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast
INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 687.59741


687.5974148361406


In [5]:
n_tot = 0.

for i in range(len(zc_edges) - 1):

    number_counts.cnc_params.update({
            "z_min": zc_edges[i],
            "z_max": zc_edges[i+1],
            "bins_edges_z": np.linspace(zc_edges[i], \
                            zc_edges[i+1], 100)
    })

    number_counts.update_params(number_counts.cosmo_params,number_counts.scal_rel_params, number_counts.cnc_params)

    number_counts.get_number_counts() #Evaluate the binned likelihood
    number_counts.get_bias_weighted_cluster_abundance() #Evaluate the binned likelihood

    n_tot_bin = number_counts.n_tot
    n_tot_bias_weighted = number_counts.n_tot_bias_weighted

    bias_mean = n_tot_bias_weighted/n_tot_bin

    print('Redshift bin edges: ', zc_edges[i], zc_edges[i+1])
    print('Average bias: ', bias_mean)
    print('Total number counts bin: ', n_tot_bin)

    n_tot += n_tot_bin

print('Total number counts: ', n_tot)

INFO - Cosmology params: {'Om0': 0.31, 'Ob0': 0.04900000000000001, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.0866006382888505e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.0013931690968152894, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast
INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 209.99297
INFO - Collecting hmf
INFO - Cosmology params: {'Om0': 0.31, 'Ob0': 0.04900000000000001, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.0866006382888505e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.0013931690968152894, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast


Redshift bin edges:  0.2 0.4
Average bias:  6.0795890430665125
Total number counts bin:  209.9929741668106


INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 191.43440
INFO - Collecting hmf
INFO - Cosmology params: {'Om0': 0.31, 'Ob0': 0.04900000000000001, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.0866006382888505e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.0013931690968152894, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast


Redshift bin edges:  0.4 0.55
Average bias:  6.889954397713498
Total number counts bin:  191.43439974252283


INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 164.50120
INFO - Collecting hmf
INFO - Cosmology params: {'Om0': 0.31, 'Ob0': 0.04900000000000001, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.0866006382888505e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.0013931690968152894, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast


Redshift bin edges:  0.55 0.7
Average bias:  7.683793115032606
Total number counts bin:  164.50120334008406


INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 121.68425
INFO - Collecting hmf


Redshift bin edges:  0.7 0.85
Average bias:  8.512504684030837
Total number counts bin:  121.68424704597201
Total number counts:  687.6128242953895


### Checking against catalog

In [6]:
file_catalog = '/lustre/scratch/data/anicola_hpc-ACT_clustering/data/DR5ClusterSearch/DR5ClusterSearch_optimalCatalog.fits'

hdulist = fits.open(file_catalog)

cat = hdulist[1].data

Number of clusters in DES footprint

In [7]:
np.sum(cat['footprint_DES'][cat['fixed_SNR'] > 5.0])

948

Check against full theory prediction

In [9]:
number_counts.cnc_params.update({
        "z_min": 0.01,
        "z_max": 3,
        "bins_edges_z": np.linspace(0.01, \
                        3, 100)
})

number_counts.update_params(number_counts.cosmo_params,number_counts.scal_rel_params, number_counts.cnc_params)

number_counts.get_number_counts() #Evaluate the binned likelihood

n_tot = number_counts.n_tot

print(n_tot)

INFO - Cosmology params: {'Om0': 0.31, 'Ob0': 0.04900000000000001, 'Ob0h2': 0.022657600000000007, 'Oc0h2': 0.12068640000000003, 'h': 0.68, 'A_s': 2.0866006382888505e-09, 'n_s': 0.965, 'm_nu': 6e-06, 'sigma_8': 0.811, 'tau_reio': 0.0561, 'w0': -1.0, 'Onu0': 0.0013931690968152894, 'N_eff': 3.046, 'k_cutoff': 100000000.0, 'ps_cutoff': 1, 'H0': 68.0, 'sigma8': 0.81}
INFO - adding dndlnM to class_sz output
INFO - computing class_szfast
INFO - computing class_szfast done
INFO - Collecting hmf
INFO - Total clusters: 929.99541


929.9954125297949
